In [1]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
from itertools import product
import os.path as osp
from scipy.optimize import leastsq
import time
from scipy.integrate import solve_ivp
from scipy.integrate import odeint
import math
import sunode
import sunode.wrappers.as_pytensor
from copy import deepcopy

# from tqdm import tqdm, trange
from tqdm.notebook import  tqdm
from IPython.display import display as print
from datetime import datetime

%config InlineBackend.figure_format = 'retina'
az.style.use("arviz-darkgrid")

In [2]:
class MyDataset(object):

    def __init__(self, dataset_path):

        self.dataset_path = dataset_path
        
        df = pd.read_csv(dataset_path)
        
        self.df = df
        self._setup()
    
    def _setup(self):
        
        df = self.df
        # 数据初步处理
        # 计算反应速率rate，初始速率固定设置为0
        for c_i, (col_name, col_sr) in enumerate(df.items()):
            if "error" in col_name or "time" in col_name or "rate" in col_name:
                continue
            rate_col_name = f"{col_name}_rate"
            rates = []
            pre_t = None
            pre_v = None
            for th, (index, value) in zip(df['time'], col_sr.items()):
                if int(index) == 0:
                    rates.append(0.0)
                    pre_t = th
                    pre_v = value
                    continue

                delta_t = th - pre_t
                delta_value = value - pre_v
                # print(col_name, index, pre_t, th, pre_v ,value)
                rates.append(delta_value / delta_t)
                pre_t = th
                pre_v = value
            df[rate_col_name] = rates

        # 准备输出值 Y
        self.cct_names = []
        for x in self.df.columns:
            if "time" in x or "error" in x or "rate" in x:
                continue
            self.cct_names.append(x)
        self.rates_names = [f"{x}_rate" for x in self.cct_names]
        self.error_names = [f"{x}-error" for x in self.cct_names]

        self.cct = self.df[self.cct_names].values
        self.rates = self.df[self.rates_names].values
        self.errors = self.df[self.error_names].values
    
    
    def set_as_sim_dataset(self, dcdt_fuc, t_eval, y0, args):
        
        y = odeint(dcdt_fuc, y0=y0, t=t_eval, args=args)
        
        # y.shape (size, 10)
        df_new = pd.DataFrame(columns=['time'] + self.cct_names)
        df_new['time'] = t_eval
        
        for c_name, col_val in zip(self.cct_names, np.transpose(y, [1,0])):
            df_new[c_name] = col_val
            df_new[f"{c_name}-error"] = 0.001
        
        self.df = df_new
        self._setup()
    
    def set_dataset(self, c_pred_df):
        
        df = deepcopy(c_pred_df)
        df['time'] = self.df['time'].values
        for c_name in self.cct_names:
            c_error_name = f"{c_name}-error"
            n_len = len(df[c_name].values)
            _error = np.repeat(0.0001, n_len)
            df[c_error_name] = _error
        self.df = df
        self._setup()


    def get_rates(self):
        return self.rates

    def get_df(self):
        return self.df

    def get_errors(self):
        return self.errors

    def get_cct(self):
        return self.cct

    def get_var_col_names(self):
        return self.cct_names, self.rates_names, self.error_names

    def get_weights(self):
        max_value = self.df[self.cct_names].describe().loc['max'].values.max()

        vars_max = self.df[self.cct_names].describe().loc['max']
        weights = (max_value / vars_max).values

        return np.array(weights)

    def get_vars_max(self):
        vars_max = self.df[self.cct_names].describe().loc['max'].values
        return vars_max


def get_format_time(f_s=None):
    haomiao = str(time.time()).split('.')[-1]
    if f_s is None:
        f_s = "%Y-%-m-%d %H:%M:%S."
        return datetime.now().strftime(f_s) + haomiao
    return datetime.now().strftime(f_s)
def plot_dataset(dataset, dataset_pred=None):
    
    df = dataset.get_df()
    cct_names, rates_names, error_names = dataset.get_var_col_names()
    
    cols = 5
    rows = math.ceil(len(cct_names) / cols)

    fig, fig_axes = plt.subplots(ncols=cols, nrows=rows, figsize=(4.2 * cols, 4 * rows), dpi=100)
    if isinstance(fig_axes, np.ndarray):
        fig_axes = fig_axes.reshape(-1)
    else:
        fig_axes = [fig_axes]

    for i, axes in enumerate(fig_axes):
        if i >= len(cct_names):
            axes.axis('off')
            continue
        
        y_name = cct_names[i]
        Y = df[y_name].values
        axes.plot(df['time'].values, Y, 'g', label=f"ob")
        axes.set_ylabel(f'cct_{y_name}')
        axes.set_xlabel(f'time(h)')

        # axes.plot(df['time'].values, df[rates_names[i]].values, '+', label=f"rate")
        
        if dataset_pred:
            _df_pred = dataset_pred.get_df()
            t_eval = _df_pred['time'].values
            axes.plot(t_eval, _df_pred[y_name].values, 'r', label=f"c(t)")
        
        
        # axes.plot(t_eval, dcdt_df[y_name].values,'g', label=f"c'(t)")

        axes.legend()
        # axes.set_title(f"{y_name}", fontsize=14)

    plt.tight_layout()
    plt.show()

In [3]:
def get_dcdt_func(k_kinetics):
    k_kinetics = np.array(k_kinetics).astype(np.uint8)
    def _dcdt_func(t, c, p):
        r1 = p.k1 * c.xN2 if k_kinetics[0] == 1 else p.k1
        r2 = p.k2 * c.xNH3 if k_kinetics[1] == 1 else p.k2
        r3 = p.k3 * c.xNO2 if k_kinetics[2] == 1 else p.k3
        r4 = p.k4 * c.xNO3 if k_kinetics[3] == 1 else p.k4
        r5 = p.k5 * c.xNO2 if k_kinetics[4] == 1 else p.k5
        r6 = p.k6 * c.xNO2 * c.xNO3 if k_kinetics[5] == 1 else p.k6
        r7 = p.k7 * c.xNO3 if k_kinetics[6] == 1 else p.k7
        r8 = p.k8 * c.xNO3 if k_kinetics[7] == 1 else p.k8
        r9 = p.k9 * c.xNH3 if k_kinetics[8] == 1 else p.k9
        r10 = p.k10 * c.xNOrg if k_kinetics[9] == 1 else p.k10
        r11 = p.k11 * c.xNOrg if k_kinetics[10] == 1 else p.k11

        
        dc_xNH3 = 2 * r1 + r7 + r10 - r2 - r6 - r9
        dc_xNO3 = r3 - r7 - r4 - r8 + r11
        dc_xNO2 = r2 + r4 - r3 - r6 - 2 * r5
        dc_xNOrg = r8 + r9 - r10 - r11
        dc_xN2 = r5 + r6 - r1

        dc_ANH3 = (2 * r1 * (c.AN2 - c.ANH3) + (c.ANO3 - c.ANH3) * r7 + (c.ANOrg - c.ANH3) * r10) / c.xNH3
        dc_ANO3 = ((c.ANO2 - c.ANO3) * r2 + (c.ANOrg - c.ANO3) * r11) / c.xNO3
        dc_ANO2 = ((c.ANH3 - c.ANO2) * r2 + (c.ANO3 - c.ANO2) * r4) / c.xNO2
        dc_ANOrg = ((c.ANO3 - c.ANOrg) * r8 + (c.ANH3 - c.ANOrg) * r9) / c.xNOrg
        dc_AN2 = ((c.ANO2 - c.AN2) * r5 + (c.ANO2 * c.ANH3 - c.AN2) * r6) / c.xN2

        # dcdts = [dc_xNH3, dc_xNO3, dc_xNO2, dc_xNOrg, dc_xN2, dc_ANH3, dc_ANO3, dc_ANO2, dc_ANOrg, dc_AN2]
        
        dcdts =  {
            'xNH3': dc_xNH3,
            'xNO3': dc_xNO3,
            'xNO2': dc_xNO2,
            'xNOrg': dc_xNOrg,
            'xN2': dc_xN2,
            'ANH3': dc_ANH3,
            'ANO3': dc_ANO3,
            'ANO2': dc_ANO2,
            'ANOrg': dc_ANOrg,
            'AN2': dc_AN2,
        }
        return dcdts
    
    return _dcdt_func

In [4]:
def dcdt_func_for_odeint(c, t, ks, k_kinetics):
    
    # print(c, t, ks, k_kinetics)
    # print()
    c_xNH3, c_xNO3, c_xNO2, c_xNOrg, c_xN2, c_ANH3, c_ANO3, c_ANO2, c_ANOrg, c_AN2 = c
    
    r1 = ks[0] * c_xN2 if k_kinetics[0] == 1 else ks[0]
    r2 = ks[1] * c_xNH3 if k_kinetics[1] == 1 else ks[1]
    r3 = ks[2] * c_xNO2 if k_kinetics[2] == 1 else ks[2]
    r4 = ks[3] * c_xNO3 if k_kinetics[3] == 1 else ks[3]
    r5 = ks[4] * c_xNO2 if k_kinetics[4] == 1 else ks[4]
    r6 = ks[5] * c_xNO2 * c_xNO3 if k_kinetics[5] == 1 else ks[5]
    r7 = ks[6] * c_xNO3 if k_kinetics[6] == 1 else ks[6]
    r8 = ks[7] * c_xNO3 if k_kinetics[7] == 1 else ks[7]
    r9 = ks[8] * c_xNH3 if k_kinetics[8] == 1 else ks[8]
    r10 = ks[9] * c_xNOrg if k_kinetics[9] == 1 else ks[9]
    r11 = ks[10] * c_xNOrg if k_kinetics[10] == 1 else ks[10]
    
    dc_xNH3 = 2 * r1 + r7 + r10 - r2 - r6 - r9
    dc_xNO3 = r3 - r7 - r4 - r8 + r11
    dc_xNO2 = r2 + r4 - r3 - r6 - 2 * r5
    dc_xNOrg = r8 + r9 - r10 - r11
    dc_xN2 = r5 + r6 - r1
    dc_ANH3 = (2 * r1 * (c_AN2 - c_ANH3) + (c_ANO3 - c_ANH3) * r7 + (c_ANOrg - c_ANH3) * r10) / c_xNH3
    dc_ANO3 = ((c_ANO2 - c_ANO3) * r2 + (c_ANOrg - c_ANO3) * r11) / c_xNO3
    dc_ANO2 = ((c_ANH3 - c_ANO2) * r2 + (c_ANO3 - c_ANO2) * r4) / c_xNO2
    dc_ANOrg = ((c_ANO3 - c_ANOrg) * r8 + (c_ANH3 - c_ANOrg) * r9) / c_xNOrg
    dc_AN2 = ((c_ANO2 - c_AN2) * r5 + (c_ANO2 * c_ANH3 - c_AN2) * r6) / c_xN2

    dcdts = [dc_xNH3, dc_xNO3, dc_xNO2, dc_xNOrg, dc_xN2, dc_ANH3, dc_ANO3, dc_ANO2, dc_ANOrg, dc_AN2]

    return np.array(dcdts)

# simulator function
def competition_model(rng, t_eval, y0,  ks, k_kinetics, size=None):
    # print(y0)
    y = odeint(dcdt_func_for_odeint, y0=y0, t=t_eval, args=(ks, k_kinetics))
    return y


In [5]:
def r2_loss(pred, y):
    r2_loss = 1 - np.square(pred - y).sum() / np.square(y - np.mean(y)).sum()
    return r2_loss


def get_model(dataset, k_kinetics, k_sigma_priors=0.01, kf_type=0):

    df = dataset.get_df()
    times = df['time'].values
    
    errors = dataset.get_errors()
    rates = dataset.get_rates()
    cct_names, rates_names, error_names = dataset.get_var_col_names()
        
    # 定义参数优化模型
    mcmc_model = pm.Model()
    ## 参数个数
    params_n = 11
    parames ={}
    
    with mcmc_model:
        for ki in range(1, params_n + 1):
            if kf_type == 0:
                p_dense = pm.HalfNormal(f"k{ki}", sigma=k_sigma_priors)
            else:
                p_dense = pm.Normal(f"k{ki}",mu=0, sigma=k_sigma_priors)
            parames[f"k{ki}"] = (p_dense, ())
    
    # parames['k_kinetics'] = np.array(k_kinetics, dtype=np.float64)
    parames['extra']=  np.zeros(1)
    
    c0 = {}
    with mcmc_model:
        for c_name in cct_names:
            _maxx = df[c_name].values.max()
            c0[f"{c_name}"] = (pm.HalfNormal(f"{c_name}_s", sigma=_maxx), ())
            # c0[f"{c_name}"] = (pm.Normal(f"{c_name}_s", mu=_meanx, sigma=s), ())
        
    
        y_hat, _, problem, solver, _, _ = sunode.wrappers.as_pytensor.solve_ivp(
            y0=c0,
            params=parames,
            rhs=get_dcdt_func(k_kinetics),
            tvals=times,
            t0=times[0],
        )
        
        sd = pm.HalfNormal('sd')
        for c_name in cct_names:
            pm.Normal(f'{c_name}', mu=y_hat[f"{c_name}"], sigma=sd, observed=df[f"{c_name}"].values)
            pm.Deterministic(f'{c_name}_mu', y_hat[f"{c_name}"])

    return mcmc_model


In [32]:
def get_predict_ks(idata_sum):
    parames_summary = az.summary(idata, round_to=10)
    ks_names = [f"k{x+1}" for x in range(11)]

    predict_ks = []
    for k_name in ks_names:
        k_v = parames_summary["mean"][k_name]
        predict_ks.append(k_v)
    return np.array(predict_ks)




In [33]:
def get_result(dataset, idata_sum):
    res_df = idata_sum
    
    predict_ks = get_predict_ks(idata_sum=idata_sum)
    

    df = dataset.get_df()
    times = df['time'].values
    
    errors = dataset.get_errors()
    rates = dataset.get_rates()
    cct_names, rates_names, error_names = dataset.get_var_col_names()
    
    
    
    predict_start = []
    
    for c_name in cct_names:
        _x = res_df['mean'][f'{c_name}_s']
        predict_start.append(_x)
    
    
    
    c_df = pd.DataFrame(columns=cct_names)
    t_n = len(times)
    for c_name in cct_names:
        c_value = []
        for i in range(t_n):
            _x = res_df['mean'][f"{c_name}_mu[{i}]"]
            c_value.append(_x)
        c_df[c_name] = c_value
    
    return predict_ks, predict_start, c_df



In [37]:

res_path_list = glob.glob("save_idata/*")
dataset = MyDataset("dataset/data.csv")
df = dataset.get_df()
cct_names, rates_names, error_names = dataset.get_var_col_names()


title_str = "k10,k_kinetics"

for c_name in cct_names:
    title_str += f",{c_name}_r2"
    
title_str += ",r2_mean,r2_all"

print(title_str)

'k10,k_kinetics,xNH3_r2,xNO3_r2,xNO2_r2,xNOrg_r2,xN2_r2,ANH3_r2,ANO3_r2,ANO2_r2,ANOrg_r2,AN2_r2,r2_mean,r2_all'

In [ ]:
import glob
import pickle
from copy import deepcopy
import os


res_path_list = glob.glob("save_idata/*")
dataset = MyDataset("dataset/data.csv")
df = dataset.get_df()
cct_names, rates_names, error_names = dataset.get_var_col_names()

res_csv = open("res_r2.csv", 'w+', encoding="utf8")

title_str = "k10,k_kinetics"

for c_name in cct_names:
    title_str += f",{c_name}_r2"
    
title_str += ",r2_mean,r2_all"
res_csv.write(f"{title_str}\n")


for index, res_file_path in enumerate(res_path_list):
    
    print(f"{index+1}/{len(res_path_list)} {res_file_path}")
    
    line_str = ""
    
    k_10 = os.path.basename(res_file_path).split('-')[0]
    k_10_2 = bin(int(k_10))[2:]
    k_kinetics = f"{k_10_2:0>11}"
    
    line_str += f"{k_10},{k_kinetics}"
    
    f_size = os.path.getsize(res_file_path)
    if f_size<1:
        continue
    
    idata = pickle.load(open(res_file_path, 'rb'))
    idata_sum = az.summary(idata, round_to=10)
    
    predict_ks, predict_start, c_df = get_result(dataset, idata_sum)
    
    r2_losses = []
    for cct_name in cct_names:
        _l = r2_loss(df[cct_name],c_df[cct_name])
        line_str += f",{_l}"
        r2_losses.append(_l)
    
    r2_all = r2_loss(df[cct_names].values, c_df.values)
    r2_mean = np.array(r2_losses).mean()
    line_str += f",{r2_mean}, {r2_all}"
    res_csv.write(f"{line_str}\n")

res_csv.close()  

'1/2048 save_idata/994-idata.dt'

'2/2048 save_idata/999-idata.dt'

'3/2048 save_idata/991-idata.dt'

'4/2048 save_idata/998-idata.dt'

'5/2048 save_idata/997-idata.dt'

...

'1972/2048 save_idata/1062-idata.dt'

'1973/2048 save_idata/1061-idata.dt'

'1974/2048 save_idata/1068-idata.dt'

'1975/2048 save_idata/106-idata.dt'

'1976/2048 save_idata/107-idata.dt'